In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
import os
import matplotlib.pyplot as plt

In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgbm

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, GaussianNoise
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


# 데이터 불러오기

In [4]:
# 데이터 불러오기
train = pd.read_csv('/home/lab21/data/train.csv', index_col=0)
train_2 = pd.read_csv('/home/lab21/data/train_x_0.2_99.8.csv', index_col=0)

test = pd.read_csv('/home/lab21/data/test.csv', index_col=0)
sample_submission = pd.read_csv('/home/lab21/data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
#x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = train_2.drop(columns=['fiberID'], axis=1)

test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [5]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [6]:
x=x.astype('float32')
test_x=test_x.astype('float32')

# Train_Test_SpIit

In [7]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, \
                                                    train_size=0.8, shuffle=True ,random_state=42)

# Scaler

In [8]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

# Variables

In [9]:
import datetime
start=datetime.datetime.now()
print(start)

2020-02-25 00:32:36.461937


# GridSearch For XGBoost

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [10]:
params= {
    'n_estimators': [50,10],
    'max_depth': [6,10],
}

In [11]:
xgb_clf = xgb.XGBClassifier()

In [ ]:
grid_cv =GridSearchCV(xgb_clf, params, cv=2, n_jobs=4)
grid_cv.fit(x_train,y_train)

In [ ]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

In [ ]:
best_params_ = grid_cv.best_params_

In [ ]:
print(best_params_)

# Predict

In [ ]:
xgb_clf = xgb.XGBClassifier(**best_params_, n_jobs=8, verbosity=0)

In [ ]:
xgb_clf.fit(x_train, y_train)

In [ ]:
print("test acc: {:.4f}".format(xgb_clf.score(x_test, y_test)))

In [ ]:
pred=xgb_clf.predict_proba(test_x)

In [ ]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)
now=datetime.datetime.now()
print(now)

# Model Predict